# Installation and Repository cloning

In [ ]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 29.3 MB/s eta 0:01:26
     ---------------------------------------- 0.0/2.5 GB 28.8 MB/s eta 0:01:27
     ---------------------------------------- 0.0/2.5 GB 29.1 MB/s eta 0:01:26
     ---------------------------------------- 0.0/2.5 GB 28.9 MB/s eta 0:01:26
     ---------------------------------------- 0.0/2.5 GB 28.8 MB/s eta 0:01:27
      --------------------------------------- 0.0/2.5 GB 29.0 MB/s eta 0:01:26
      --------------------------------------- 0.0/2.5 GB 28.9 MB/s eta 0:01:26
      --------------------------------------- 0.0/2.5 GB 29.0 MB/s eta 0:01:26
      --------------------------------------- 0.1/2.5 GB 28.9 MB/s eta 0:01:25
      --------------------------------------- 0.1/2.5 GB 28.9 MB/s eta 0:01:25
     - -------------------------------------- 0.1/2.5 GB 28.9 MB/s eta 0:01:25
 

  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-yi2awuf6'


  Using cached matplotlib-3.9.3-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.2-cp39-cp39-win_amd64.whl.metadata (168 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
Using cached matplotlib-3.9.3-cp39-cp39-win_amd64.whl (7.8 MB)
Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl (211 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.55.2-cp39-cp39-win_amd64.whl (2.2 MB)
Using cached importlib_resources-6.4.5-py3-none-any.whl (36 kB)
Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl (55 kB)
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)
  Using cached foolbox-3.3.4-py3-none-any.whl.metadata (7.3 kB)
  Using cached

## Import Libraries

In [1]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-

cuda


## Load and Prepare Adversarial Model

In [2]:
model = 'Wang2023Better_WRN-28-10' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10
net, art_net, fb_net, alias = utils.get_model(modelname=model, norm='L2') # specify the norm in case of robustbench models

c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\robustbench\utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = tor

## Load the dataset

In [3]:
splitsize = 500 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset_split=splitsize)

Files already downloaded and verified


## Test Accuracy

In [4]:
utils.test_accuracy(net, xtest, ytest)


Accuracy of the test set is: 95.400%



## Parameters 

In [5]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon = 12, 
                        eps_iter = 0.15,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations_fast_attacks = 10, 
                        max_iterations_slow_attacks = 10, 
                        verbose=False)

Hyperparameter Sweep Experiment

In [6]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack'
hyperparameter_range=[0.1,0.2,0.5,1.0,2.0]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, 
                                               range=hyperparameter_range, 
                                               attack_type=attack_type)

[0.1, 0.2, 0.5, 1.0, 2.0]
		-------------- Hyperparameter Sweep for Attack: exp_attack: learning_rate = 0.1 ----------------



c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\adversarial_attack\exp_attack.py:255: RuntimeWarning: invalid value encountered in log
  v_val = np.where(abc>=15.0,np.log(abc)-np.log(np.log(abc))+np.log(np.log(abc))/np.log(abc), lambertw( np.exp(abc), k=0).real)/b-a


20 images done. Current Adversarial Accuracy: 90.0%
60 images done. Current Adversarial Accuracy: 86.66666666666667%
80 images done. Current Adversarial Accuracy: 87.5%
100 images done. Current Adversarial Accuracy: 88.0%
120 images done. Current Adversarial Accuracy: 88.33333333333333%
140 images done. Current Adversarial Accuracy: 87.14285714285714%
160 images done. Current Adversarial Accuracy: 86.25%
180 images done. Current Adversarial Accuracy: 85.55555555555556%
200 images done. Current Adversarial Accuracy: 85.0%
220 images done. Current Adversarial Accuracy: 84.54545454545455%
240 images done. Current Adversarial Accuracy: 84.16666666666667%


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
#Attack comparison
attack_types = [#'fast_gradient_method', #bounded-full
                #'projected_gradient_descent', #bounded-full
                #'pgd_early_stopping', #not-bounded
                #'auto_projected_gradient_descent', #bounded-full
                #'deep_fool', #not-bounded
                #'brendel_bethge', #bounded-min
                #'carlini_wagner_l2', #not-bounded
                'elastic_net', #not-bounded
                'exp_attack',
                #'exp_attack_smooth',
                #'elastic_net_L1_rule', #not-bounded
                #'elastic_net_L1_rule_higher_beta', #not-bounded
                #'ART_AutoAttack', #bounded-full
                #'original_AutoAttack', #bounded-full
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)

['elastic_net', 'exp_attack']
		-------------------------- Processing Attack: elastic_net --------------------------

20 images done. Current Adversarial Accuracy: 55.0%
60 images done. Current Adversarial Accuracy: 66.66666666666667%
80 images done. Current Adversarial Accuracy: 67.5%
100 images done. Current Adversarial Accuracy: 69.0%
120 images done. Current Adversarial Accuracy: 68.33333333333333%
140 images done. Current Adversarial Accuracy: 67.85714285714286%
160 images done. Current Adversarial Accuracy: 67.5%
180 images done. Current Adversarial Accuracy: 68.33333333333333%
200 images done. Current Adversarial Accuracy: 67.5%
240 images done. Current Adversarial Accuracy: 69.58333333333333%
260 images done. Current Adversarial Accuracy: 70.38461538461539%
280 images done. Current Adversarial Accuracy: 70.71428571428571%
300 images done. Current Adversarial Accuracy: 71.33333333333333%
340 images done. Current Adversarial Accuracy: 70.29411764705883%
360 images done. Current A